In [103]:
import requests
from bs4 import BeautifulSoup
import requests.packages.urllib3
import jsons

from tinydb import TinyDB, Query
from tinydb import where
import threading
requests.packages.urllib3.disable_warnings()

rootFolder=os.getcwd()+"/"
baseFolder=rootFolder+"temp/"   
try:
    os.mkdir(baseFolder) 
except:
    pass

logFile=rootFolder+'logger.log'
refreshUrl='https://onlineplus.mofidonline.com/Home/Default/page-1'
refreshMinutes=15

chromeWebDriverPath = '/usr/lib/chromium-browser/chromedriver'


cookieFilename='MofidOnlineCookieFile'
cookieFile = baseFolder+cookieFilename+'.json'
chromeProfilePath=baseFolder+'ChromeProfile'
orderFile = rootFolder+'orderList.json'
lockFile=cookieFile+".lock"
cookieFileIndexInput="-one"

headers = {
              "origin":"https://onlineplus.mofidonline.com"\
              ,"sec-fetch-dest":"empty"\
              ,"user-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/80.0.3987.163 Chrome/80.0.3987.163 Safari/537.36"\
              ,"accept":"*/*"\
              ,"sec-fetch-site":"cross-site"\
              ,"sec-fetch-mode": "cors"\
              ,"referer":"https://onlineplus.mofidonline.com/Home/Default/page-1"\
              ,"accept-encoding":"gzip, deflate, br"\
              ,"accept-language":"en-US,en;q=0.9,fa;q=0.8"\
            }
def getInfoSymbol(symbolISIN):
    try:
        localCookieFilename = baseFolder+cookieFilename+cookieFileIndexInput+'.json'
        symbolISIN = "IRO1KVIR0001"
        url='https://core.tadbirrlc.com//StockFutureInfoHandler.ashx?{"Type":"getLightSymbolInfoAndQueue","la":"fa","nscCode":"'+symbolISIN+'"}&jsoncallback='

        response = requests.get(url, headers=headers, verify = False, timeout=(10, 20))

        stockFutureInfoHandler = (jsons.loads(response.content))
        info={}
        info['isin'] = stockFutureInfoHandler['symbolinfo']['nc']
        info['lastTradePrice'] = stockFutureInfoHandler['symbolinfo']['ltp']
        info['closePrice'] = stockFutureInfoHandler['symbolinfo']['cp']
        info['highRangePrice'] = stockFutureInfoHandler['symbolinfo']['ht']
        info['lowRangePrice'] = stockFutureInfoHandler['symbolinfo']['lt']
        info['yesterdayClosePrice'] = stockFutureInfoHandler['symbolinfo']['pcp']
        info['DateTime'] = stockFutureInfoHandler['symbolinfo']['ltd']
        info['id'] = info['isin'] + ' ' + info['DateTime']
        info['Date'] = info['DateTime'][0:0+10]
        info['Time'] = info['DateTime'][13:13+8]
        info['closePointRelativePrice'] = stockFutureInfoHandler['symbolinfo']['cpv']
        info['closePointRelativePercent'] = stockFutureInfoHandler['symbolinfo']['cpvp']
        info['lastPointRelativePrice'] = stockFutureInfoHandler['symbolinfo']['lpv']
        info['lastPointRelativePercent'] = stockFutureInfoHandler['symbolinfo']['lpvp']


        bestBuyQuantity = float(stockFutureInfoHandler['symbolqueue']['Value'][0]['BestBuyQuantity'])+1
        noBestBuy = float(stockFutureInfoHandler['symbolqueue']['Value'][0]['NoBestBuy']) + 1
        bestSellQuantity = float(stockFutureInfoHandler['symbolqueue']['Value'][0]['BestSellQuantity'])+1
        noBestSell = float(stockFutureInfoHandler['symbolqueue']['Value'][0]['NoBestSell']) + 1
        buyToToSellPower = float(bestBuyQuantity/noBestBuy)/(bestSellQuantity/noBestSell)
        info['buyToToSellPower1Row']=buyToToSellPower

        bestBuyQuantity = 0
        noBestBuy = 0
        bestSellQuantity = 0
        noBestSell = 0
        for counter in range(0,4):
            bestBuyQuantity += float(stockFutureInfoHandler['symbolqueue']['Value'][counter]['BestBuyQuantity'])+1
            noBestBuy += float(stockFutureInfoHandler['symbolqueue']['Value'][counter]['NoBestBuy']) + 1
            bestSellQuantity += float(stockFutureInfoHandler['symbolqueue']['Value'][counter]['BestSellQuantity'])+1
            noBestSell += float(stockFutureInfoHandler['symbolqueue']['Value'][counter]['NoBestSell']) + 1

        buyToToSellPower5Rows = float(bestBuyQuantity/noBestBuy)/(bestSellQuantity/noBestSell)
        info['buyToToSellPower5Rows']=buyToToSellPower5Rows

        # sendOrderurl='https://onlineplus.mofidonline.com/Handlers/GetAccountRemain.ashx'
        # response = requests.get(sendOrderurl, headers=headers, verify = False, timeout=(10, 20))
        # accountRemain = (jsons.loads(response.content))
        # print(accountRemain)


        #TODO: try except is not exist
        url='https://core.tadbirrlc.com//AlmasDataHandler.ashx?{"Type":"getIndInstTrade","la":"Fa","nscCode":"'+symbolISIN+'","ZeroIfMarketIsCloesed":true}&jsoncallback='
        response = requests.get(url, headers=headers, verify = False, timeout=(10, 20))
        IndInstTrade = (jsons.loads(response.content))

        realBuyVolume = int(IndInstTrade['IndBuyVolume'])
        realBuyNumber = int(IndInstTrade['IndBuyNumber'])
        realSellVolume = int(IndInstTrade['IndSellVolume'])
        realSellNumber = int(IndInstTrade['IndSellNumber'])

        legalBuyVolume = int(IndInstTrade['InsBuyVolume'])
        legalBuyNumber = int(IndInstTrade['InsBuyNumber'])
        legalSellVolume = int(IndInstTrade['InsSellVolume'])
        legalSellNumber = int(IndInstTrade['InsSellNumber'])
        realLegalDate = IndInstTrade['day']

        info['realBuyVolume'] = realBuyVolume
        info['realBuyNumber'] =realBuyNumber 
        info['realSellVolume'] = realSellVolume
        info['realSellNumber'] = realSellNumber

        info['legalBuyVolume'] = legalBuyVolume 
        info['legalBuyNumber'] = legalBuyNumber
        info['legalSellVolume'] = legalSellVolume
        info['legalSellNumber'] = legalSellNumber

        buyToSellPowerToday = float(realBuyVolume/realBuyNumber)/float(realSellVolume/realSellNumber)
        buyToSellLegalPowerToday = float(legalBuyVolume)/float(legalSellVolume)
        legalBuyPercent=legalBuyVolume/(legalBuyVolume+realBuyVolume)
        legalSellPercent=legalSellVolume/(legalSellVolume+realSellVolume)

        info['buyToSellPowerToday'] = buyToSellPowerToday
        info['buyToSellLegalPowerToday'] = buyToSellLegalPowerToday
        info['legalBuyPercent'] = legalBuyPercent
        info['legalSellPercent'] = legalSellPercent
    except Exception as err:
        #TODO:correct this after log!!
        pass
    return info
getInfoSymbol("IRO1TAMN0001")   


{'isin': 'IRO1KVIR0001',
 'lastTradePrice': 46120,
 'closePrice': 46540,
 'highRangePrice': 50300,
 'lowRangePrice': 45520,
 'yesterdayClosePrice': 47910,
 'DateTime': '1399/04/04 - 12:50:30',
 'id': 'IRO1KVIR0001 1399/04/04 - 12:50:30',
 'Date': '1399/04/04',
 'Time': '12:50:30',
 'closePointRelativePrice': -1370,
 'closePointRelativePercent': -2.86,
 'lastPointRelativePrice': -1790,
 'lastPointRelativePercent': -3.74,
 'buyToToSellPower1Row': 0.15183077542104959,
 'buyToToSellPower5Rows': 0.18994136947985585,
 'realBuyVolume': 15247279,
 'realBuyNumber': 2455,
 'realSellVolume': 11165919,
 'realSellNumber': 2906,
 'legalBuyVolume': 1357403,
 'legalBuyNumber': 6,
 'legalSellVolume': 5438763,
 'legalSellNumber': 8,
 'buyToSellPowerToday': 1.6163744899923214,
 'buyToSellLegalPowerToday': 0.24957936207185347,
 'legalBuyPercent': 0.08174820812587678,
 'legalSellPercent': 0.32754394212427557}

In [79]:
4/0

ZeroDivisionError: division by zero

In [118]:
import sqlite3
def loadInfoSymbolListFromDB():
    conn = sqlite3.connect(r"InfoSymbol.db")
    c = conn.cursor()
    c.execute("SELECT * FROM InfoSymbols WHERE isin='IRO1TAMN0001' ")
    data=c.fetchall()
    conn.close()
    return data
data = loadInfoSymbolListFromDB()
print(data)

[('IRO1TAMN0001 1399/04/04 - 12:29:59', 'IRO1TAMN0001', 22950, 22540, 22950, 20770, 21860, '1399/04/04 - 12:29:59', '1399/04/04', '12:29:59', 680.0, 3.11, 1090.0, 4.99, 3.2209462261588335, 3.2259547947410474, 99493211, 11810, 156967372, 30813, 76446727, 41, 18972566, 41, 1.6537434855311468, 4.029329875568755, 0.43450468306974166, 0.10783547053426835)]
